# 모듈 불러오기

In [2]:
import numpy as np
import pandas as pd

from scipy import stats

import warnings
import os
import gc
warnings.filterwarnings('ignore')


print(pd.options.display.max_rows)
print(os.listdir('../dataset'))

60
['sample_submission.csv', 'test_identity.csv', 'test_transaction.csv', 'train_identity.csv', 'train_transaction.csv']


# 함수 정의

## 메모리 사용량 감소 함수

데이터프레임의 컬럼별 최대값과 최소값을 이용하여 컬럼의 타입을 변경

In [1]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## 테이블 요약 함수

In [4]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

# 데이터 불러오기

In [6]:
path = '../dataset/'
train_identity = pd.read_csv(path + 'train_identity.csv')
train_transaction = pd.read_csv(path + 'train_transaction.csv')

train_identity = reduce_mem_usage(train_identity)
train_transaction = reduce_mem_usage(train_transaction)

print("Train: Identitiy Dataset Shape: ", train_identity.shape)
print("Train: Transaction Dataset Shape: ", train_transaction.shape)

Mem. usage decreased to 25.86 Mb (42.7% reduction)
Mem. usage decreased to 542.35 Mb (69.4% reduction)
Train: Identitiy Dataset Shape:  (144233, 41)
Train: Transaction Dataset Shape:  (590540, 394)


In [9]:
path = '../dataset/'
test_identity = pd.read_csv(path + 'test_identity.csv')
test_transaction = pd.read_csv(path + 'test_transaction.csv')

test_identity = reduce_mem_usage(test_identity)
test_transaction = reduce_mem_usage(test_transaction)

print("test: Identitiy Dataset Shape: ", test_identity.shape)
print("test: Transaction Dataset Shape: ", test_transaction.shape)

Mem. usage decreased to 25.44 Mb (42.7% reduction)
Mem. usage decreased to 472.59 Mb (68.9% reduction)
test: Identitiy Dataset Shape:  (141907, 41)
test: Transaction Dataset Shape:  (506691, 393)


In [15]:
submission = pd.read_csv(path + 'sample_submission.csv')

# 데이터 설명 및 요약

## 데이터 설명

[Data Description (Details and Discussion)](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203)을 통해 주어진 데이터 정의서는 다음과 같다.

### 거래(Transaction) 데이터

- TransactionDT: 지정된 시점으로부터 지난 시간(실제 타임스탬프는 아님)
- TransactionAMT: 거래 결제 금액(USD)
- ProductCD: 거래에 대한 제품 코드
- card1 - card6: 카드의 종류, 카드의 카테고리, 카드 발급 은행, 발급 국가 등 결제 카드 정보
- addr: 주소
- dist: 거리
- P_ and (R_) emalildomain: 구매자(Purchaser) 및 수령인(Recipient)의 이메일 도메인
- C1 - C14: 결제 카드와 관련된 주소가 몇 개인지 계산.(실제 의미는 마스킹된다.)
- D1 - D15: 이전 거래까지의 시간
- M1 - M5: 카드 소유자와 주소의 매치 여부
- Vxxx: 주최측인 Vesta가 기타 기업 관계에 대한 순위 및 빈도를 포함한 다양한 변수들.

여기에서 카테고리 변수는 다음과 같다.
- ProductCD
- card1 - card6
- addr1, addr2
- Pemaildomain, Remaildomain
- M1 - M9

## 데이터 요약

정의한 함수를 이용하여 데이터 요약 테이블을 생성한다. .describe()와 달리, 컬럼별 고유값, Null값을 확인할 수 있다.

In [14]:
resumetable(train_transaction[train_transaction.columns[:25]])

Dataset Shape: (590540, 25)


,Name,dtypes,Missing,Uniques,First Value,Second Value,Third Value,Entropy
0,TransactionID,int32,0,590540,2987000,2987001,2987002,19.17
1,isFraud,int8,0,2,0,0,0,0.22
2,TransactionDT,int32,0,573349,86400,86401,86469,19.11
3,TransactionAmt,float16,0,8195,68.5,29,59,8.10
4,ProductCD,object,0,5,W,W,W,1.28
5,card1,int16,0,13553,13926,2755,4663,9.97
6,card2,float16,8933,500,NaN,404,490,6.32
7,card3,float16,1565,114,150,150,150,0.68
8,card4,object,1577,4,discover,mastercard,visa,1.09
9,card5,float16,4259,119,142,102,166,2.66


# TransactionDT